# PyTorch Intro
PyTorch Official Tutorial: https://pytorch.org/tutorials/

This file will be uploaded to the Homework repository.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import transforms

import matplotlib.pyplot as plt
import numpy as np

import pdb

In [ ]:
import loader # a module (.py file) we created. Read for more information

basic_transform = transforms.Compose([transforms.ToTensor()])

batch_size = 32

trainloader, validloader = loader.get_data_loader(basic_transform, 
                                                  basic_transform, 
                                                  batch_size)


In [ ]:
# [32, 3, 30, 30] = [batch size, channels, height, width]
for x, y in trainloader:
    print(x.shape)
    print(y.shape)
    print(y)
    break

# vis
for i in range(4):
    plt.imshow(np.transpose(x[i,:], (1,2,0))) # 30 x 30 x 3
    plt.show()


In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        
        # define the layers
        # kernel size = 3 means (3,3) kernel
        # rgb -> 3 -> in channel
        # number of feature maps = 16
        # number of kernel parameters = out_channels x in_channels x kernel_size x kernel_size
        self.l1 = nn.Conv2d(kernel_size=3, in_channels=3, out_channels=16)
        
        # MaxPool2d, AvgPool2d. 
        # The first 2 = 2x2 kernel size, 
        # The second 2 means the stride=2
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2) 
        
        self.l2 = nn.Conv2d(kernel_size=3, in_channels=16, out_channels=32)
        
        # FC layer
        self.fc1 = nn.Linear(32 * 6 * 6, 5) # IMPORTANT ! 
        # 32 (channels) * heigth in out (6) * width in out (6)
        
    def forward(self, x):
        # define the data flow through the deep learning layers
        
        # x = self.l1(x)
        # x = F.relu(x)
        # x = self.pool(x)
        # These three lines above will be equivalent to the following line
        
        x = self.pool(F.relu(self.l1(x))) # bs x 16 x 14 x 14. Width 30 ->(conv) 28 ->(pool) 14
        x = self.pool(F.relu(self.l2(x))) # bs x 32 x 6 x 6. Width 14 -> (conv) ? -> (pool) ?
        # print(x.shape)
        x = x.reshape(-1, 32*6*6) # [bs x 1152]# CRUCIAL!
        # print(x.shape)
        x = self.fc1(x)
        return x

In [ ]:
m = CNN()
pred = m(x) # x is one batch of data [32, 3, 30, 30]
# pred is [32, 5] because we have 32 images and 5 classes
print(pred.shape)

In [ ]:
print(pred)
print(torch.softmax(pred, dim=1))

# Training

In [ ]:
criterion = nn.CrossEntropyLoss()
num_epoches = ? # give me a number from 2-5
import tqdm

import torch.optim as optim


USE_CUDA = torch.cuda.is_available()

if USE_CUDA:
    m = m.cuda()

In [ ]:
for epoch_id in range(num_epoches):
    optimizer = optim.SGD(m.parameters(), lr=0.01 * 0.95 ** epoch_id)
    for x, y in tqdm.tqdm(trainloader):
        if USE_CUDA:
            x, y = x.cuda(), y.cuda()
        optimizer.zero_grad() # clear (reset) the gradient for the optimizer
        pred = m(x) # forward pass, will call forward(self, x) function.
        loss = criterion(pred, y)
        loss.backward() # Backward pass: calculating the gradient
        optimizer.step() # Updating the Parameters in backpropagation: optimize the model

# Testing

In [ ]:
all_gt = []
all_pred = []

for x, y in tqdm.tqdm(validloader):
    if USE_CUDA:
        x, y = x.cuda(), y.cuda()
    all_gt += list(y.detach().cpu().numpy().reshape(-1))
    pred = torch.argmax(m(x), dim=1)
    all_pred += list(pred.detach().cpu().numpy().reshape(-1))
    
    

In [ ]:
print(all_gt)
print(all_pred)

In [ ]:
acc = np.sum(np.array(all_gt) == np.array(all_pred)) / len(all_gt)
print("Accuracy is:", acc)